In [1]:
!pip install nengo nengo-loihi

In [2]:
import nengo
import nengo_loihi

In [3]:
import torch
val_path = 'final_project/val_dataset.pth'
train_path = 'final_project/train_dataset.pth'
# Load the model
test_val_dataset_loaded= torch.load(val_path)
train_dataset_loaded= torch.load(train_path)

In [24]:
import numpy as np

def load_npz_events(file_path):
    """
    Load and return the content of an .npz file containing event data.
    :param file_path: The path to the .npz file.
    :return: A dictionary with keys 't', 'x', 'y', 'p'.
    """
    try:
        # Assuming the file format is not zip
        data = np.load(file_path, allow_pickle=True)
        return {key: data[key] for key in ['t', 'x', 'y', 'p']}
    except Exception as e:
        print(f"Error loading the file {file_path}: {e}")
        return None

In [5]:
import tqdm
import math
import numpy as np
def split_to_train_test_set(train_ratio: float, origin_dataset: torch.utils.data.Dataset, num_classes: int, random_split: bool = False):
    '''
    :param train_ratio: split the ratio of the origin dataset as the train set
    :type train_ratio: float
    :param origin_dataset: the origin dataset
    :type origin_dataset: torch.utils.data.Dataset
    :param num_classes: total classes number, e.g., ``10`` for the MNIST dataset
    :type num_classes: int
    :param random_split: If ``False``, the front ratio of samples in each classes will
            be included in train set, while the reset will be included in test set.
            If ``True``, this function will split samples in each classes randomly. The randomness is controlled by
            ``numpy.random.seed``
    :type random_split: int
    :return: a tuple ``(train_set, test_set)``
    :rtype: tuple
    '''
    label_idx = []
    for i in range(num_classes):
        label_idx.append([])

    for i, item in enumerate(tqdm.tqdm(origin_dataset)):

        y = item[1]

        if isinstance(y, np.ndarray) or isinstance(y, torch.Tensor):
            y = y.item()
        label_idx[y].append(i)
    train_idx = []
    test_idx = []
    if random_split:
        for i in range(num_classes):
            np.random.shuffle(label_idx[i])

    for i in range(num_classes):
        pos = math.ceil(label_idx[i].__len__() * train_ratio)
        train_idx.extend(label_idx[i][0: pos])
        test_idx.extend(label_idx[i][pos: label_idx[i].__len__()])

    return torch.utils.data.Subset(origin_dataset, train_idx), torch.utils.data.Subset(origin_dataset, test_idx)


In [5]:
#RUN to USE THE WHOLE DATASET
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np

class CustomNPZDataset(Dataset):
    def __init__(self, root_dir, extensions=('.npz',)):
        self.file_paths = []
        self.labels = []
        self.classes = os.listdir(root_dir)
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        for cls in self.classes:
            cls_folder = os.path.join(root_dir, cls)
            if os.path.isdir(cls_folder):
                for file in os.listdir(cls_folder):
                    if file.endswith(extensions):
                        self.file_paths.append(os.path.join(cls_folder, file))
                        self.labels.append(self.class_to_idx[cls])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        data = load_npz_events(file_path)  # Your custom loader function
        label = self.labels[idx]
        return data, label

# Replace 'root_dir' with your dataset directory
dataset = CustomNPZDataset(root_dir='extracted_content/content/CIFAR/events_np')

# Splitting the dataset
train_size = int(0.7 * len(dataset))  # 70% of data for training
test_size = len(dataset) - train_size  # Remaining 30% for testing/validation
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


# Optionally create DataLoader instances for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [6]:
import numpy as np

test_file_path = 'extracted_content/content/CIFAR/events_np/airplane/cifar10_airplane_975.npz'  # Replace with the actual file path
try:
    data = np.load(test_file_path)
    print("File loaded successfully.")
    print("Keys in the file:", list(data.keys()))
except Exception as e:
    print("Error loading the file:", e)


File loaded successfully.
Keys in the file: ['t', 'x', 'y', 'p']


In [20]:
#run to USE 5 images per category 
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np

class CustomNPZDataset(Dataset):
    def __init__(self, root_dir, extensions=('.npz',), samples_per_class=5): #USE 5 IMAGES PER CLASS
        self.file_paths = []
        self.labels = []
        self.classes = os.listdir(root_dir)
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        for cls in self.classes:
            cls_folder = os.path.join(root_dir, cls)
            if os.path.isdir(cls_folder):
                cls_files = [file for file in os.listdir(cls_folder) if file.endswith(extensions)]
                cls_files = cls_files[:samples_per_class]  # Select only a certain number of samples
                for file in cls_files:
                    self.file_paths.append(os.path.join(cls_folder, file))
                    self.labels.append(self.class_to_idx[cls])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        data = load_npz_events(file_path)
        if data is None:
            # Handle the case where data couldn't be loaded
            # For example, you can skip this item, return an empty data, etc.
            pass
        label = self.labels[idx]
        return data, label

# Create an instance of the dataset
dataset = CustomNPZDataset(root_dir='extracted_content/content/CIFAR/events_np')
print(dataset[0])

# Splitting the dataset
train_size = int(0.7 * len(dataset))  # Adjust the ratio as needed
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


# create DataLoader instances for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Attempting to load file: extracted_content/content/CIFAR/events_np/automobile/cifar10_automobile_207.npz
Error loading the file extracted_content/content/CIFAR/events_np/automobile/cifar10_automobile_207.npz: File is not a zip file
(None, 0)


In [8]:
def print_original_data_structure(event_data):
    print("Original Data Structure:")
    for key in event_data:
        print(f"{key}: shape = {event_data[key].shape}, sample = {event_data[key][:10]}")


event_data = np.load('extracted_content/content/CIFAR/events_np/airplane/cifar10_airplane_979.npz')
print_original_data_structure(event_data)


Original Data Structure:
t: shape = (148123,), sample = [ 0  2  7 11 16 21 25 29 34 38]
x: shape = (148123,), sample = [117 113 112 115 114 124 125 127 126 120]
y: shape = (148123,), sample = [60 60 60 60 60 60 60 60 60 60]
p: shape = (148123,), sample = [1 1 1 1 1 1 1 1 1 1]


In [9]:
def get_data(dataset, batch_size=32, split_ratios=(0.7, 0.15, 0.15), bin_size=10000):
    """
    Load and preprocess CIFAR-10 DVS data, then split it into train, validation, and test sets.
    :param dataset: The CustomNPZDataset instance.
    :param batch_size: Batch size for DataLoader.
    :param split_ratios: Tuple of ratios for splitting dataset (train, validation, test).
    :param bin_size: Size of each time bin in microseconds for temporal binning.
    :return: DataLoaders for train, validation, and test sets.
    """
    # Calculate sizes for train, validation, and test sets
    total_size = len(dataset)
    train_size = int(split_ratios[0] * total_size)
    val_size = int(split_ratios[1] * total_size)
    test_size = total_size - train_size - val_size

    # Randomly split dataset
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

    # DataLoader for each set
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    return train_loader, val_loader, test_loader

def collate_fn(batch):
    """
    Custom collate function to apply preprocessing steps to each batch.
    """
    processed_batch = []
    labels = []
    for data, label in batch:
        # Apply preprocessing functions
        data = preprocess_cifar10_dvs_data(data)
        data = temporal_binning(data, bin_size=10000)  # Adjust bin size as needed
        data = [normalize_data(frame, method='min-max') for frame in data]
        processed_batch.append(data)
        labels.append(label)
    return torch.stack(processed_batch), torch.tensor(labels)

# Create an instance of the dataset
dataset = CustomNPZDataset(root_dir='extracted_content/content/CIFAR/events_np')

# Get DataLoaders
train_loader, val_loader, test_loader = get_data(dataset)


In [10]:
def preprocess_cifar10_dvs_data(event_data, img_size=(128, 128)):
    """
    Preprocess CIFAR-10 DVS event data into a format suitable for the SNN.
    :param event_data: CIFAR-10 DVS data (npz file contents).
    :param img_size: Tuple of the image size (height, width).
    :return: Preprocessed data.
    """
    height, width = img_size
    processed_data = np.zeros((height, width, 2))  # Assuming 2 channels for polarity

    for t, x, y, p in zip(event_data['t'], event_data['x'], event_data['y'], event_data['p']):
        if 0 <= x < width and 0 <= y < height:
            processed_data[y, x, int(p)] = 1  # Set spike at (x, y) for the given polarity

    return processed_data.flatten()  # Flatten the array to match the input layer dimensions

def temporal_binning(event_data, bin_size, img_size=(128, 128)):
    """
    Temporally bin the CIFAR-10 DVS event data.
    :param event_data: CIFAR-10 DVS data (npz file contents).
    :param bin_size: Size of each time bin in microseconds.
    :param img_size: Tuple of the image size (height, width).
    :return: List of binned data.
    """
    max_time = np.max(event_data['t'])
    num_bins = int(np.ceil(max_time / bin_size))
    height, width = img_size
    binned_data = [np.zeros((height, width, 2)) for _ in range(num_bins)]

    for t, x, y, p in zip(event_data['t'], event_data['x'], event_data['y'], event_data['p']):
        if 0 <= x < width and 0 <= y < height:
            bin_index = int(t // bin_size)
            binned_data[bin_index][y, x, int(p)] = 1

    return [bin_data.flatten() for bin_data in binned_data]

def normalize_data(data, method='min-max'):
    """
    Normalize the input data.
    :param data: The data to be normalized (expected as a flattened array).
    :param method: The method of normalization ('min-max' or 'standard').
    :return: Normalized data.
    """
    if method == 'min-max':
        data_min = np.min(data)
        data_max = np.max(data)
        return (data - data_min) / (data_max - data_min)
    elif method == 'standard':
        data_mean = np.mean(data)
        data_std = np.std(data)
        return (data - data_mean) / data_std
    else:
        raise ValueError("Unknown normalization method.")


In [11]:
def print_processed_data_structure(processed_data):
    print("Processed Data Structure:")
    if isinstance(processed_data, list):  # For temporally binned data
        print(f"Number of bins: {len(processed_data)}")
        for i, bin_data in enumerate(processed_data[:3]):  # Print first few bins
            print(f"Bin {i}: shape = {bin_data.shape}, sample = {bin_data[:10]}")
    else:  # For flattened data
        print(f"Shape = {processed_data.shape}, sample = {processed_data[:10]}")

#Apply preprocessing and then print structure
processed_data = preprocess_cifar10_dvs_data(event_data)
print_processed_data_structure(processed_data)

#For temporal binning
binned_data = temporal_binning(event_data, bin_size=10000)
print_processed_data_structure(binned_data)

# For normalization
normalized_data = normalize_data(processed_data, method='min-max')
print_processed_data_structure(normalized_data)


Processed Data Structure:
Shape = (32768,), sample = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Processed Data Structure:
Number of bins: 135
Bin 0: shape = (32768,), sample = [1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
Bin 1: shape = (32768,), sample = [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Bin 2: shape = (32768,), sample = [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Processed Data Structure:
Shape = (32768,), sample = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [12]:
import nengo
from nengo_loihi import Simulator

# Define the network structure
with nengo.Network() as net:
    # Loihi compatible LIF neuron parameters
    lif_params = {
        'tau_rc': 0.02,  # membrane time constant
        'tau_ref': 0.002, # refractory period
        # other parameters as needed
    }

    # Input layer - adapt size as needed
    input_layer = nengo.Ensemble(n_neurons=128*128*2, dimensions=1, neuron_type=nengo.LIF(**lif_params))

    # Hidden layers
    hidden_layer_1 = nengo.Ensemble(512, 1, neuron_type=nengo.LIF(**lif_params))
    hidden_layer_2 = nengo.Ensemble(512, 1, neuron_type=nengo.LIF(**lif_params))

    # Output layer
    output_layer = nengo.Ensemble(10, 1, neuron_type=nengo.LIF(**lif_params))

    # Connections
    nengo.Connection(input_layer, hidden_layer_1)
    nengo.Connection(hidden_layer_1, hidden_layer_2)
    nengo.Connection(hidden_layer_2, output_layer)


In [26]:
def preprocess_and_split_data(dataset, bin_size=10000, normalization_method='min-max', split_ratios=(0.7, 0.15, 0.15)):
    """
    Preprocess the dataset and split it into training, validation, and testing sets.
    """
    # Preprocess all data
    preprocessed_data = []
    for i in range(len(dataset)):
        data, label = dataset[i]
        processed = preprocess_cifar10_dvs_data(data)
        binned = temporal_binning(processed, bin_size=bin_size)
        normalized = [normalize_data(frame.flatten(), method=normalization_method) for frame in binned]
        preprocessed_data.append((normalized, label))
    
    # Calculate split sizes
    total_size = len(preprocessed_data)
    train_size = int(split_ratios[0] * total_size)
    val_size = int(split_ratios[1] * total_size)
    test_size = total_size - train_size - val_size

    # Split the data
    train_data = preprocessed_data[:train_size]
    val_data = preprocessed_data[train_size:train_size + val_size]
    test_data = preprocessed_data[train_size + val_size:]

    return train_data, val_data, test_data

# Create an instance of the dataset
dataset = CustomNPZDataset(root_dir='extracted_content/content/CIFAR/events_np')

# Use the modified get_data() with the dataset instance
original_train_data, original_val_data, original_test_data = preprocess_and_split_data(dataset)



Error loading the file extracted_content/content/CIFAR/events_np/automobile/cifar10_automobile_207.npz: File is not a zip file


TypeError: 'NoneType' object is not subscriptable

In [ ]:

# Simulator configuration for Loihi
with Simulator(net, precompute=True) as sim:
    # Training loop
    for epoch in range(num_epochs):
        for inputs, targets in train_data:
            # Run the network with the current inputs
            for input_frame in inputs:
                sim.run(input_frame)
            # Apply learning and error computation here

    # Validation
    for inputs, targets in val_data:
        for input_frame in inputs:
            sim.run(input_frame)
        # Evaluate the network's performance on the validation set

    # Testing
    for inputs, targets in test_data:
        for input_frame in inputs:
            sim.run(input_frame)
        # Evaluate the network's performance on the test set

In [ ]:
# Simulator configuration for Loihi
with Simulator(net, precompute=True) as sim:
    train_data, val_data, test_data = get_data()

    # Training loop
    for epoch in range(num_epochs):
        for inputs, targets in train_data:
            # Preprocess each input
            inputs = preprocess_cifar10_dvs_data(inputs)
            inputs = temporal_binning(inputs, bin_size=100)  # Example bin size
            inputs = normalize_data(inputs.flatten(), method='min-max')
            sim.run(inputs)

            # Define learning connections with a learning rule
            learning_conn = nengo.Connection(hidden_layer_1, hidden_layer_2, 
                                             learning_rule_type=nengo.PES(learning_rate=1e-4))

            # Error population for learning
            error = nengo.Ensemble(n_neurons=10, dimensions=1)
            nengo.Connection(output_layer, error)
            nengo.Connection(error, learning_conn.learning_rule)


     # Validation
    for inputs, targets in val_data:
        # Preprocess inputs (already done in the DataLoader)
        sim.run(inputs)
        # Evaluate the network's performance on the validation set
        # Code for evaluation goes here

    # Testing
    for inputs, targets in test_data:
        # Preprocess inputs (already done in the DataLoader)
        sim.run(inputs)
        # Evaluate the network's performance on the test set
        # Code for evaluation goes here


In [ ]:
import nengo
from nengo_loihi import Simulator
import numpy as np
import time


# Load and preprocess data
train_data, val_data, test_data = load_data()  # You need to define this function
preprocessed_train_data = preprocess_data(train_data)
preprocessed_val_data = preprocess_data(val_data)



# Simulator configuration for Loihi
with Simulator(net, precompute=True) as sim:
    # Training and validation loop
    for epoch in range(1, episodes + 1):
        start_time = time.time()
        
        # Training loop
        for input, label in preprocessed_train_data:
            sim.run(input)
            # Implement learning here (if applicable)

        # Validation loop
        latency_sum = 0
        num_batches = 0
        throughput_start_time = time.time()
        for input, label in preprocessed_val_data:
            batch_start_time = time.time()
            sim.run(input)
            batch_end_time = time.time()
            latency_batch = batch_end_time - batch_start_time
            latency_sum += latency_batch
            num_batches += 1

        throughput_end_time = time.time()
        average_latency = latency_sum / num_batches if num_batches > 0 else 0
        throughput_time = throughput_end_time - throughput_start_time
        throughput = num_batches / throughput_time if throughput_time > 0 else 0

        epoch_latency.append(average_latency)
        epoch_throughputs.append(throughput)

        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Epoch {epoch} completed in {elapsed_time} seconds')

# Final calculations and output
print(f"Average Latency over all epochs: {np.mean(epoch_latency)}")
print(f"Average Throughput over all epochs: {np.mean(epoch_throughputs)}")
